In [1]:
import numpy as np
import os
from tqdm import tqdm_notebook
import h5py
import time
from sklearn.model_selection import train_test_split
import gc
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Activation

from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard

In [2]:
seed = 0
num_categories = 20
num_epochs = 500
batch_size = 64

In [3]:
def scheduler(epoch):
    if epoch < 200:
        return 0.01
    if epoch < 400:
        return 0.005
    return 0.001

# Callback for tensorboard
tb_cb = TensorBoard(log_dir='./age_cnn', histogram_freq=0)

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                           save_weights_only=True,
                                           verbose=1,
                                           epochs_pr_checkpoint=50)

# callback for learning rate 
change_lr_cb = LearningRateScheduler(scheduler)

cbks = [cp_cb, tb_cb, change_lr_cb]

In [4]:
def categorize(ages,categories=20):
    ages_copy=np.array(ages,copy=True)
    for i in range(len(ages)):
        if ages_copy[i] < 0:
            ages_copy[i] = 0
        elif ages_copy[i] > 99:
            ages_copy[i] = categories+1
        else:
            ages_copy[i] = ages_copy[i]//(100/categories)+1
    categorical = to_categorical(ages_copy)
    return categorical

In [5]:
def load_data():
    with h5py.File('imdb_preprocesed_1.hdf5', 'r') as f:
        i1_ages = np.array(f.get('ages'))
        i1_images = np.array(f.get('images'))

    print("Part 1 done!")

    with h5py.File('imdb_preprocesed_2.hdf5', 'r') as f:
        i2_ages = np.array(f.get('ages'))
        i2_images = np.array(f.get('images'))

    print("Part 2 done!")

    with h5py.File('wiki_preprocesed.hdf5', 'r') as f:
        w_ages = np.array(f.get('ages'))
        w_images = np.array(f.get('images'))

    print("Load done!\nCreating dataset")
    gc.collect()
    
    images = np.concatenate([i1_images,i2_images, w_images])
    ages = np.concatenate([i1_ages, i2_ages, w_ages])
    
    ages = categorize(ages, categories=num_categories)
    
    x_train, x_test, y_train, y_test = train_test_split(images, ages, test_size = 0.1, random_state=seed)
    print("All done!")
    gc.collect()
    return x_train, x_test, y_train, y_test

In [6]:
# load and split the data
x_train, x_test, y_train, y_test = load_data()

# split the training in train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=seed)

Part 1 done!
Part 2 done!
Load done!
Creating dataset
All done!


In [7]:
def createConvModel(inputDimensions, outputDimensions):
    model = Sequential()
    
    # Convblock 1
    model.add(Conv2D(64, kernel_size=3, padding='same', input_shape=inputDimensions))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, kernel_size=3, padding='same', input_shape=inputDimensions))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Convblock 2
    model.add(Conv2D(128, kernel_size=3, padding='same', input_shape=inputDimensions))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(128, kernel_size=3, padding='same', input_shape=inputDimensions))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Dense finale
    model.add(Flatten())
    
    #model.add(Dense(100, activation="relu"))
    model.add(Dense(outputDimensions, activation="softmax"))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
x_test.shape

In [8]:
model = createConvModel(x_train[0].shape, y_train.shape[1]) 

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0

In [ ]:
model.fit(x_train,
          y_train,
          validation_data=(x_val, y_val),
          epochs=num_epochs,
          callbacks=cbks,
          batch_size=batch_size)


Train on 423670 samples, validate on 47075 samples
Epoch 1/500
423552/423670 [============================>.] - ETA: 0s - loss: 2.6671 - accuracy: 0.1650
Epoch 00001: saving model to training_1/cp.ckpt
423670/423670 [==============================] - 120s 282us/sample - loss: 2.6670 - accuracy: 0.1650 - val_loss: 2.3356 - val_accuracy: 0.1689
Epoch 2/500
423552/423670 [============================>.] - ETA: 0s - loss: 2.3228 - accuracy: 0.1736
Epoch 00002: saving model to training_1/cp.ckpt
423670/423670 [==============================] - 113s 267us/sample - loss: 2.3228 - accuracy: 0.1736 - val_loss: 2.3170 - val_accuracy: 0.1766
Epoch 3/500
423616/423670 [============================>.] - ETA: 0s - loss: 2.2845 - accuracy: 0.1843
Epoch 00003: saving model to training_1/cp.ckpt
423670/423670 [==============================] - 113s 267us/sample - loss: 2.2845 - accuracy: 0.1843 - val_loss: 2.2667 - val_accuracy: 0.1897
Epoch 4/500
423616/423670 [============================>.] - ETA: 0

In [ ]:
# loss history for simple cnn model
cnn_history = model.history.history
plt.figure(figsize=(12, 8))
plt.plot(cnn_history["val_loss"], label="Validation loss")
plt.plot(cnn_history["loss"], label="Training loss")
plt.legend();

In [ ]:
# accuracy history
plt.figure(figsize=(12, 8))
plt.plot(cnn_history["val_acc"], label="Validation accuracy")
plt.plot(cnn_history["acc"], label="Training accuracy")
plt.legend();

## Warning, do not run until the model is done!!!

In [ ]:
# final metrics for the model computed on the test set
final_model_metrics = model.evaluate(x_test, y_test)